In [ ]:
!pip install py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 943.9 kB/s eta 0:00:00


In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.7 MB/s eta 0:00:00


In [ ]:
!pip install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
!pip -q install langchain openai tiktoken yfinance


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.272
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, google-api-core, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Understanding OpenAI Functions

### Simple example

In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    }
]

### Setting up YFinance


In [ ]:
import yfinance as yf

def get_stock_price(symbol,tool_input=None):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'][0], 2)

print(get_stock_price('AAPL'))



176.38


### Setting up tools

In [ ]:
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import Tool
from langchain.tools import BaseTool

search = DuckDuckGoSearchRun()
# # # defining a single tool
DUCKSEARCH = Tool(
         name = "search",
          func=search.run,
          description="useful for when you need to answer questions about current events. You should ask targeted questions"
   )


In [ ]:
from pydantic import BaseModel, Field

class StockPriceCheckInput(BaseModel):
    """Input for Stock price check."""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")


In [ ]:
class StockPriceTool(BaseTool):
    name = "get_stock_ticker_price"
    description = "Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"

    def _run(self, stockticker: str):
        # print("i'm running")
        price_response = get_stock_price(stockticker)

        return price_response

    def _arun(self, stockticker: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockPriceCheckInput


In [ ]:
StockPriceTool = Tool(
    name='GetStockTickerprice',
    func= get_stock_price,
     description="Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"
 )

## LangChain doing it Manualy

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage, FunctionMessage

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo-0613")

In [ ]:
from langchain.tools import MoveFileTool, format_tool_to_openai_function

In [ ]:
{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'title': 'FileMoveInput',
                'description': 'Input for MoveFileTool.',
                'type': 'object',
                'properties': {'source_path': {'title': 'Source Path',
                                               'description': 'Path of the file to move',
                                               'type': 'string'},
                               'destination_path': {'title': 'Destination Path',
                                               'description': 'New path for the moved file',
                                               'type': 'string'}},
                'required': ['source_path', 'destination_path']}}

{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'title': 'FileMoveInput',
  'description': 'Input for MoveFileTool.',
  'type': 'object',
  'properties': {'source_path': {'title': 'Source Path',
    'description': 'Path of the file to move',
    'type': 'string'},
   'destination_path': {'title': 'Destination Path',
    'description': 'New path for the moved file',
    'type': 'string'}},
  'required': ['source_path', 'destination_path']}}

In [ ]:
MoveFileTool().args_schema.schema()

{'title': 'FileMoveInput',
 'description': 'Input for MoveFileTool.',
 'type': 'object',
 'properties': {'source_path': {'title': 'Source Path',
   'description': 'Path of the file to move',
   'type': 'string'},
  'destination_path': {'title': 'Destination Path',
   'description': 'New path for the moved file',
   'type': 'string'}},
 'required': ['source_path', 'destination_path']}

In [ ]:
tools = [StockPriceTool]
functions = [format_tool_to_openai_function(t) for t in tools]

In [ ]:
functions[0]

{'name': 'GetStockTickerprice',
 'description': 'Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API',
 'parameters': {'properties': {'__arg1': {'title': '__arg1',
    'type': 'string'}},
  'required': ['__arg1'],
  'type': 'object'}}

In [ ]:
ai_message = model.predict_messages([HumanMessage(content='What is the price of Google stock')], functions=functions)

In [ ]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'GetStockTickerprice', 'arguments': '{\n  "__arg1": "GOOGL"\n}'}}, example=False)

In [ ]:
ai_message.additional_kwargs['function_call']

{'name': 'GetStockTickerprice', 'arguments': '{\n  "__arg1": "GOOGL"\n}'}

In [ ]:
import json
_args = json.loads(ai_message.additional_kwargs['function_call'].get('arguments'))
_args

{'__arg1': 'GOOGL'}

In [ ]:
tool_result = tools[0](_args)
tool_result

129.78

In [ ]:
FunctionMessage(name='get_stock_ticker_price',content=tool_result)

FunctionMessage(content='129.78', additional_kwargs={}, name='get_stock_ticker_price')

In [ ]:
final_message = model.predict_messages([HumanMessage(content='What is the price of Google stock'),
                                  ai_message,
                                  FunctionMessage(name='get_stock_ticker_price',content=tool_result),
                                  ], functions=functions)

In [ ]:
final_message

AIMessage(content='The price of Google stock (GOOGL) is $129.78.', additional_kwargs={}, example=False)

## Putting it together as an Agent

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [ ]:
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)

In [ ]:
open_ai_agent.run("What is the price of Google stock?")



> Entering new AgentExecutor chain...

Invoking: `GetStockTickerprice` with `GOOGL`




129.78

The current price of Google stock (GOOGL) is $129.78.

> Finished chain.


'The current price of Google stock (GOOGL) is $129.78.'

## Multiple Tools

In [ ]:
from datetime import datetime, timedelta

def get_price_change_percent(symbol, days_ago):
    ticker = yf.Ticker(symbol)

    # Get today's date
    end_date = datetime.now()

    # Get the date N days ago
    start_date = end_date - timedelta(days=days_ago)

    # Convert dates to string format that yfinance can accept
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')

    # Get the historical data
    historical_data = ticker.history(start=start_date, end=end_date)

    # Get the closing price N days ago and today's closing price
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]

    # Calculate the percentage change
    percent_change = ((new_price - old_price) / old_price) * 100

    return round(percent_change, 2)

# Use the function
print(get_price_change_percent('AAPL', 30))  # for 30 days ago


-9.19


In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

def calculate_performance(symbol, days_ago):
    ticker = yf.Ticker(symbol)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_ago)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    historical_data = ticker.history(start=start_date, end=end_date)
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]
    percent_change = ((new_price - old_price) / old_price) * 100
    return round(percent_change, 2)

def get_best_performing(stocks, days_ago):
    best_stock = None
    best_performance = None
    for stock in stocks:
        try:
            performance = calculate_performance(stock, days_ago)
            if best_performance is None or performance > best_performance:
                best_stock = stock
                best_performance = performance
        except Exception as e:
            print(f"Could not calculate performance for {stock}: {e}")
    return best_stock, best_performance

stocks = ['AAPL', 'MSFT', 'GOOG']
days_ago = 90  # change as desired

best_stock, best_performance = get_best_performing(stocks, days_ago)
print(f"The best performing stock over the past {days_ago} days is {best_stock} with a performance of {best_performance}%")


The best performing stock over the past 90 days is GOOG with a performance of 4.64%


### Make the Tools

In [ ]:
from typing import List


class StockChangePercentageCheckInput(BaseModel):
    """Input for Stock ticker check. for percentage check"""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")
    days_ago: int = Field(..., description="Int number of days to look back")

class StockPercentageChangeTool(BaseTool):
    name = "get_price_change_percent"
    description = "Useful for when you need to find out the percentage change in a stock's value. You should input the stock ticker used on the yfinance API and also input the number of days to check the change over"

    def _run(self, stockticker: str, days_ago: int):
        price_change_response = get_price_change_percent(stockticker, days_ago)

        return price_change_response

    def _arun(self, stockticker: str, days_ago: int):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockChangePercentageCheckInput


# the best performing

class StockBestPerformingInput(BaseModel):
    """Input for Stock ticker check. for percentage check"""

    stocktickers: List[str] = Field(..., description="Ticker symbols for stocks or indices")
    days_ago: int = Field(..., description="Int number of days to look back")

class StockGetBestPerformingTool(BaseTool):
    name = "get_best_performing"
    description = "Useful for when you need to the performance of multiple stocks over a period. You should input a list of stock tickers used on the yfinance API and also input the number of days to check the change over"

    def _run(self, stocktickers: List[str], days_ago: int):
        price_change_response = get_best_performing(stocktickers, days_ago)

        return price_change_response

    def _arun(self, stockticker: List[str], days_ago: int):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockBestPerformingInput
from typing import List
from pydantic import BaseModel, Field
from typing import Optional, Type
import math
import random  # Import random to generate random allocation percentages
class InvestmentPlanInput(BaseModel):
    """Input for creating an investment plan"""

    timeframe_years: float = Field(..., description="How many years do you plan to save for buying a house?")
    risk_tolerance: str = Field(..., description="Your risk tolerance: 'high' or 'low'")
    investment_amount: float = Field(..., description="How much money are you planning to invest towards buying a house?")

class InvestmentPlanTool(BaseTool):
    name = "create_investment_plan"
    description = "Generate an investment plan based on your goals, risk tolerance, and investment amount."

    def _run(self, timeframe_years: int, risk_tolerance: str, investment_amount: float):
        # Generate stock tickers with allocation percentages based on risk tolerance
        selected_stocks, allocation_percentages = generate_stock_tickers_with_allocation(risk_tolerance)

        # Calculate allocation amounts based on investment amount and allocation percentages
        allocation_amounts = [investment_amount * percentage / 100 for percentage in allocation_percentages]

        # Create an investment plan
        investment_plan = {
            "Client Information": {
                "Full Name": "",
                "Date of Birth": "",
                "Contact Information": "",
                "Marital Status": "",
                "Dependents": "",
            },
            "Financial Goals": [
                {
                    "Goal": "",
                    "Target Amount": 0,
                    "Time Horizon": "",
                    "Risk Tolerance": "",
                },
                {
                    "Goal": "",
                    "Target Amount": 0,
                    "Time Horizon": "",
                    "Risk Tolerance": "",
                },
                {
                    "Goal": "",
                    "Target Amount": 0,
                    "Time Horizon": "",
                    "Risk Tolerance": "",
                },
            ],
            "Risk Tolerance Assessment": risk_tolerance,
            "Current Financial Situation": {
                "Income": 0,
                "Expenses": 0,
                "Total Assets": 0,
                "Total Liabilities": 0,
                "Net Worth": 0,
            },
            "Tax Considerations": {
                "Current Tax Bracket": 0,
                "Tax-Advantaged Accounts": "",
                "Tax-Efficient Investment Strategies": "",
            },
            "Investment Portfolio Strategy": {
                "Asset Allocation": {
                    "% Stocks": 0,
                    "% Bonds": 0,
                    "% Other": 0,
                },
                "Diversification": "",
                "Liquidity Needs": "",
                "Investment Knowledge": "",
            },
            "Economic and Market Analysis": {
                "Current Economic Conditions": "",
                "Market Trends": "",
                "Interest Rate Environment": "",
            },
            "Costs and Fees": {
                "Investment Management Fees": 0,
                "Trading Costs": 0,
                "Tax Implications": "",
            },
            "Ethical and Social Values": "",
            "Emergency Fund": {
                "Current Emergency Fund Balance": 0,
                "Recommended Emergency Fund": 0,
            },
            "Insurance Coverage": {
                "Health Insurance": "",
                "Life Insurance": "",
                "Disability Insurance": "",
            },
            "Review and Monitoring Plan": {
                "Frequency of Portfolio Review": "",
                "Criteria for Portfolio Rebalancing": "",
                "Communication Plan": "",
            },
            "Exit Strategy": {

                "Under what circumstances should investments be sold or adjusted?": "",
                "How will you handle market downturns or changes in client goals?": "",
            },
            "Additional Notes": "",
            "Next Steps": "Based on the information provided, outline specific investment recommendations and strategies tailored to the client's goals, risk tolerance, and financial situation.",
            "Disclaimer": "Include a disclaimer outlining that this plan is for informational purposes only and not a guarantee of investment performance. Encourage the client to consult with a qualified financial advisor.",
        }

        # Fill in the investment plan with actual data
        investment_plan["Financial Goals"][0]["Goal"] = "Goal 1"
        investment_plan["Financial Goals"][0]["Target Amount"] = 100000
        investment_plan["Financial Goals"][0]["Time Horizon"] = "5 years"
        investment_plan["Financial Goals"][0]["Risk Tolerance"] = risk_tolerance

        # Fill in the rest of the data...

        return investment_plan

    def _arun(self, timeframe_years: int, risk_tolerance: str, investment_amount: float):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = InvestmentPlanInput


# Define a function to generate stock tickers with allocation percentages
def generate_stock_tickers_with_allocation(risk_tolerance: str):
    # Replace this with your logic to generate stock tickers and allocation percentages
    indian_stocks_high_risk = ['RELIANCE', 'TCS', 'HDFCBANK', 'INFY', 'ICICIBANK', 'HINDUNILVR', 'KOTAKBANK', 'ITC']
    indian_stocks_low_risk = ['ITC', 'HDFCBANK', 'TCS', 'INFY']

    if risk_tolerance.lower() == 'high':
        selected_stocks = indian_stocks_high_risk
        allocation_percentages = [random.uniform(0, 1) for _ in selected_stocks]
    elif risk_tolerance.lower() == 'low':
        selected_stocks = indian_stocks_low_risk
        allocation_percentages = [random.uniform(0, 1) for _ in selected_stocks]
    else:
        raise ValueError("Invalid risk tolerance. Use 'high' or 'low'.")

    total_allocation = sum(allocation_percentages)
    allocation_percentages = [percentage / total_allocation * 100 for percentage in allocation_percentages]

    return selected_stocks, allocation_percentages
import pdfplumber
from typing import List, Optional, Type
from pydantic import BaseModel, Field

class Expense(BaseModel):
    name: str
    amount: float

class BudgetInput(BaseModel):
    """Input for creating a budget plan"""
    income: float = Field(..., description="Your monthly income")
    expenses: List[Expense] = Field(..., description="List of monthly expenses")

class BudgetPlannerTool(BaseTool):
    name = "create_budget_plan"
    description = "Generate a budget plan based on your income, expenses, and financial statements from a PDF ."

    def _run(self, income: float, expenses: List[Expense]):
        # Calculate the remaining budget based on income and expenses
        remaining_budget = income - sum(expense['amount'] for expense in expenses)

        # Create the budget plan template with filled-in information
       # Create the budget plan dictionary
        budget_plan = {
    "income": income,
    "expenses":[
    {
        "Expense Name": expense['name'],
        "Amount": f"${expense['amount']}",
    }
    for expense in expenses
],
    "total_fixed_expenses": sum(expense['amount'] for expense in expenses[0:6]),
    "total_variable_expenses": None,  # Fill in the actual values for variable expenses
    "savings_and_debt_repayment": {
        "savings": None,  # Fill in the actual value for savings
        "debt_repayment": None,  # Fill in the actual value for debt repayment
    },
    "investments": {
        "investment_contributions": None,  # Fill in the actual value for investment contributions
    },
    "taxes": {
        "estimated_tax_payments": None,  # Fill in the actual value for estimated tax payments
    },
    "one_time_and_irregular_expenses": {
        "vehicle_maintenance": None,  # Fill in the actual value for vehicle maintenance
        "gifts": None,  # Fill in the actual value for gifts
        "vacation": None,  # Fill in the actual value for vacation
        "home_repairs": None,  # Fill in the actual value for home repairs
        "other_irregular_expenses": None,  # Fill in the actual value for other irregular expenses
    },
    "entertainment_and_discretionary_spending": {
        "entertainment_hobbies": None,  # Fill in the actual value for entertainment/hobbies
        "discretionary_spending": None,  # Fill in the actual value for discretionary spending
    },
    "review_and_adjust": {
        "actual_income": income,
        "actual_expenses": None,  # Fill in the actual value for actual expenses
        "difference": remaining_budget,
    },
    "financial_goals": {
        "goal_1": None,  # Fill in the actual value for goal 1
        "goal_2": None,  # Fill in the actual value for goal 2
        "goal_3": None,  # Fill in the actual value for goal 3
    },
    "contingency": {
        "contingency_fund": None,  # Fill in the actual value for the contingency fund
    },
    "budgeting_tools": {
        "budgeting_software_apps_used": None,  # Fill in the actual value for budgeting software/apps used
    },
    "long_term_planning": {
        "long_term_financial_goals": None,  # Fill in the actual value for long-term financial goals
    },
    "financial_education": {
        "financial_books_websites_resources": None,  # Fill in the actual value for financial books/websites/resources
    }
}



        # Print the budget plan template
        return budget_plan

    args_schema: Optional[Type[BaseModel]] = BudgetInput



In [ ]:
from typing import List, Optional, Type
from pydantic import BaseModel, Field

class Debt(BaseModel):
    name: str
    amount: float
    interest_rate: float

class DebtManagementInput(BaseModel):
    """Input for creating a debt management plan"""
    monthly_payment: float = Field(..., description="Your monthly payment capacity")
    debts: List[Debt] = Field(..., description="List of debts")

class DebtPlanItem(BaseModel):
    name: str
    monthly_payment: float
    interest_payment: float
    principal_payment: float
    remaining_debt: float

class DebtManagementPlannerTool(BaseTool):
    name = "create_debt_management_plan"
    description = "Generate a debt management plan based on your monthly payment capacity and debts. Also, mention how to pay the remaining debt."

    def _run(self, monthly_payment: float, debts: List[Debt]) -> dict:
        debt_plan = []
        remaining_payment = monthly_payment

        for debt in debts:
            # Replace debt.something with debt['something']
            name = debt['name']
            amount = debt['amount']
            interest_rate = debt['interest_rate']

            interest_payment = amount * (interest_rate / 100)
            principal_payment = min(remaining_payment, monthly_payment - interest_payment)
            remaining_payment -= principal_payment

            debt_plan.append(DebtPlanItem(
                name=name,
                monthly_payment=monthly_payment,
                interest_payment=interest_payment,
                principal_payment=principal_payment,
                remaining_debt=amount - principal_payment,
            ))

        # Create a dictionary structure matching the template
        debt_management_plan = {
            "Personal Information": {
                "Name": "",
                "Address": "",
                "Phone Number": "",
                "Email": "",
            },
            "Financial Goals": {
                "Short-term (e.g., 6 months)": "",
                "Medium-term (e.g., 1-2 years)": "",
                "Long-term (e.g., 5 years or more)": "",
            },
            "Assessment of Current Debt": [
                {
                    "Creditor": debt['name'],
                    "Balance": f"${debt['amount']}",
                    "Interest Rate": f"{debt['interest_rate']}%",
                    "Minimum Payment": "",
                }
                for debt in debts
            ],
            "Monthly Budget": {
                "Total Monthly Income": "",
                "Total Monthly Expenses": "",
                "Disposable Income (Income - Expenses)": "",
            },
            "Debt Repayment Plan": {
                "Priority (High-interest first or Smallest balance first)": "",
                "Debt Repayment Strategy (e.g., Debt Snowball, Debt Avalanche)": "",
                "Monthly Debt Repayment Budget": f"${monthly_payment}",
            },
            "Negotiation and Consolidation": [
                "",
                "",
                "",
            ],
            "Emergency Fund": {
                "Goal Amount": "",
                "Progress": "",
            },
            "Additional Notes": [
                "",
                "",
            ],
            "Review and Adjustments": [
                "",
                "",
            ],
            "Professional Assistance": "Yes / No",
            "Legal Considerations": "Yes / No",
        }

        # Merge the two dictionaries
        merged_plan = {"DebtPlan": debt_plan, "DebtManagementPlan": debt_management_plan}
        return merged_plan

    args_schema: Optional[Type[BaseModel]] = DebtManagementInput


In [ ]:
from datetime import date
class TodayDateInput(BaseModel):
    """Not needed"""

class TodayDateTool(BaseTool):
    name = "get_today_date"
    description = "A tool for getting today's date."

    def _run(self):
        return date.today()
    args_schema: Optional[Type[BaseModel]] = TodayDateInput


In [ ]:
tools = [StockPriceTool,StockPercentageChangeTool(), StockGetBestPerformingTool(),InvestmentPlanTool(), BudgetPlannerTool(),DebtManagementPlannerTool(),TodayDateTool(),DUCKSEARCH]

functions = [format_tool_to_openai_function(t) for t in tools]

In [ ]:
tools

[Tool(name='GetStockTickerprice', description='Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<function get_stock_price at 0x7b04518ed990>, coroutine=None),
 StockPercentageChangeTool(name='get_price_change_percent', description="Useful for when you need to find out the percentage change in a stock's value. You should input the stock ticker used on the yfinance API and also input the number of days to check the change over", args_schema=<class '__main__.StockChangePercentageCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False),
 StockGetBestPerformingTool(name='get_best_performing', description='Useful for when you need to the performance of multiple stocks over a period. You should in

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")

In [ ]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
)

In [ ]:
agent.run("""income=40,000, bank deposit interest= 5000, expense: house rent, food, travelling micellanious, now i need savings plan with which i can save 10,000 per month taxes 5000 and fixed expense 5000 and also repay my personal loan of 1,00,000 of interest rate 10.47%, give a 1 year plan """)



> Entering new AgentExecutor chain...



Invoking: `create_budget_plan` with `{'income': 40000, 'expenses': [{'name': 'House Rent', 'amount': 15000}, {'name': 'Food', 'amount': 5000}, {'name': 'Travelling', 'amount': 3000}, {'name': 'Miscellaneous', 'amount': 2000}]}`


{'income': 40000.0, 'expenses': [{'Expense Name': 'House Rent', 'Amount': '$15000.0'}, {'Expense Name': 'Food', 'Amount': '$5000.0'}, {'Expense Name': 'Travelling', 'Amount': '$3000.0'}, {'Expense Name': 'Miscellaneous', 'Amount': '$2000.0'}], 'total_fixed_expenses': 25000.0, 'total_variable_expenses': None, 'savings_and_debt_repayment': {'savings': None, 'debt_repayment': None}, 'investments': {'investment_contributions': None}, 'taxes': {'estimated_tax_payments': None}, 'one_time_and_irregular_expenses': {'vehicle_maintenance': None, 'gifts': None, 'vacation': None, 'home_repairs': None, 'other_irregular_expenses': None}, 'entertainment_and_discretionary_spending': {'entertainment_hobbies': None, 'discretionary_spending': None}, 'review_and_adjust': {'actua

KeyboardInterrupt: ignored

In [ ]:

from langchain.prompts import MessagesPlaceholder

agent_kwargs = {
    "prompt_templates": [
        "Here is the conversation history:\n{memory}\n\n"
        "Given the tools {tools} and the conversation history, "
        "answer the following question:\n{input}"
    ]
}

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,
    agent_kwargs=agent_kwargs,
    verbose=True
)

In [ ]:
agent.run(input="use these stocks to make a investment plan for my previous stocks with investment amount of 1000 rupees" )

In [ ]:
open_ai_agent.run("Which stock out of Google, Meta and MSFT has performed best over the past 3 months?")

In [ ]:
open_ai_agent.run("How much has MSFT's stock gone up over the past 3 months?")

In [ ]:
open_ai_agent.run("1.buying house,2.long term,low,6000 rupees make a investment plan")

In [ ]:
import requests

# Your bot token and base URL
base_url = "https://api.telegram.org/botYOUR_BOT_TOKEN_HERE"

def read_msg(offset):
    parameters = {
        "offset": offset
    }
    resp = requests.get(base_url + "/getUpdates", params=parameters)
    data = resp.json()

    for result in data["result"]:
        if "photo" in result["message"]:
            send_response(result["message"]["chat"]["id"])

    if data["result"]:
        return data["result"][-1]["update_id"] + 1

def send_response(chat_id):

    parameters = {
        "chat_id": chat_id,
        "text": "Okay, I received your image file!"
    }
    resp = requests.get(base_url + "/sendMessage", params=parameters)

offset = 0
while True:
    offset = read_msg(offset)

In [ ]:
import os
import sys
import requests
# If you are using a Jupyter Notebook, uncomment the following line.
# %matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from io import BytesIO
import json
# Function to extract and format text from the JSON data
def extract_table(data):
    table = []
    for region in data['regions']:
        for line in region['lines']:
            row = ' '.join([word['text'] for word in line['words']])
            table.append(row)
    return table

def image_to_text(image_url):
  # Add your Computer Vision key and endpoint to your environment variables.
  subscription_key = '978f18b0a6d74541890e34ff3cb33594'
  endpoint = 'https://texttoimageapi.cognitiveservices.azure.com/'

  ocr_url = endpoint + "vision/v3.1/ocr"
  params = {'language': 'unk', 'detectOrientation': 'true'}

  # Fetch the image from the URL
  response = requests.get(image_url)
  response.raise_for_status()
  image_data = response.content

  # Set Content-Type to octet-stream
  headers = {'Ocp-Apim-Subscription-Key': subscription_key, 'Content-Type': 'application/octet-stream'}

  # Make the OCR request
  response = requests.post(ocr_url, headers=headers, params=params, data=image_data)

  data = json.loads(response.text)



  # Extract and format the table
  formatted_table = extract_table(data)
  return formatted_table



In [ ]:
import requests

# Your bot token and base URL
base_url = "https://api.telegram.org/bot6374687953:AAFBTrbQbM447fumbPjgB4XPAJ9Ai_UY8CM"

def read_msg(offset):
    parameters = {
        "offset": offset
    }
    resp = requests.get(base_url + "/getUpdates", params=parameters)
    data = resp.json()

    for result in data["result"]:
        if 'text' in result['message']:
          send_msg(result["message"]["chat"]["id"],result["message"]["text"])
        if "photo" in result["message"]:
            received_image_id = result["message"]["photo"][-1]["file_id"]

            image_file_url = get_image_file_path(received_image_id)
            image_to_text(image_file_url)
            send_img_msg(result["message"]["chat"]["id"], image_to_text(image_file_url))

    if data["result"]:
        return data["result"][-1]["update_id"] + 1

def get_image_file_path(received_image_id):
    parameters = {
        "file_id": received_image_id
    }
    resp = requests.get(base_url + "/getFile", params=parameters)
    data = resp.json()
    if data["ok"]:
        file_path = data["result"]["file_path"]
        # Construct the full URL to the image file
        image_file_url = f"https://api.telegram.org/file/bot6374687953:AAFBTrbQbM447fumbPjgB4XPAJ9Ai_UY8CM/{file_path}"
        return image_file_url
    return None

def send_msg(chat_id, text):
    response=str(agent.run(str(text)))
    parameters = {
        "chat_id": chat_id,
        "text": str(response)
    }
    resp = requests.get(base_url + "/sendMessage", params=parameters)

def send_img_msg(chat_id, text):
    print(str(text))
    response=str(agent.run("consider all in rupees [describe the data]"+str(text)))
    parameters = {
        "chat_id": chat_id,
        "text": str(response)
    }
    resp = requests.get(base_url + "/sendMessage", params=parameters)

offset = 0
while True:
    offset = read_msg(offset)




> Entering new AgentExecutor chain...

Invoking: `GetStockTickerprice` with `{'stockticker': 'HDFCBALADV.NS'}`




ERROR:yfinance:HDFCBALADV.NS: No data found, symbol may be delisted


IndexError: ignored

In [ ]:
''